In [28]:
import re
import string
import pandas as pd

In [57]:
df = pd.read_csv('./data/germeval18/germeval2018.training.txt', header=None, sep='\t')
df = df[pd.notnull(df[1])]
df.head()

,0,1,2
0,"@corinnamilborn Liebe Corinna, wir würden dich...",OTHER,OTHER
1,@Martin28a Sie haben ja auch Recht. Unser Twee...,OTHER,OTHER
2,@ahrens_theo fröhlicher gruß aus der schönsten...,OTHER,OTHER
3,@dushanwegner Amis hätten alles und jeden gewä...,OTHER,OTHER
4,@spdde kein verläßlicher Verhandlungspartner. ...,OFFENSE,INSULT


In [58]:
df = df.reindex(columns=[1, 0]).T.reset_index(drop=True).T
df.head()

,0,1
0,OTHER,"@corinnamilborn Liebe Corinna, wir würden dich..."
1,OTHER,@Martin28a Sie haben ja auch Recht. Unser Twee...
2,OTHER,@ahrens_theo fröhlicher gruß aus der schönsten...
3,OTHER,@dushanwegner Amis hätten alles und jeden gewä...
4,OFFENSE,@spdde kein verläßlicher Verhandlungspartner. ...


In [59]:
lookup = {'OTHER': 0,
          'OFFENSE': 1}

offenses = df[0].tolist()
offenses = [lookup[offense] for offense in offenses]
df[0] = offenses

df.head()

,0,1
0,0,"@corinnamilborn Liebe Corinna, wir würden dich..."
1,0,@Martin28a Sie haben ja auch Recht. Unser Twee...
2,0,@ahrens_theo fröhlicher gruß aus der schönsten...
3,0,@dushanwegner Amis hätten alles und jeden gewä...
4,1,@spdde kein verläßlicher Verhandlungspartner. ...


In [60]:
lines = df[1].tolist()

In [61]:
# lower-case all upper-case words
for idx, line in enumerate(lines):
    words = line.split()
    lines[idx] = ' '.join([word.lower() if word.isupper() \
                            else word for word in words])

In [62]:
# remove double occurences
for idx, line in enumerate(lines):
    line = re.sub(r'\?+', '?', line, re.UNICODE)
    line = re.sub(r'\!+', '!', line, re.UNICODE)
    line = re.sub(r'\,+', ',', line, re.UNICODE)
    line = re.sub(r'\*+', '*', line, re.UNICODE)
    line = re.sub(r'\.[\.]+', ' … ', line, re.UNICODE)
    lines[idx] = re.sub(r'\-+', '-', line, re.UNICODE)

In [63]:
# drop hyphes and asterisks not between characters
_re1 = re.compile(r'(?<!\w)\*+(?!\w)|(?<!\w)\-+(?!\w)', re.UNICODE)
for idx, line in enumerate(lines):
    lines[idx] = _re1.sub('', line)

In [64]:
# replace & with 'and'
_re1 = re.compile(r'\&', re.UNICODE)
for idx, line in enumerate(lines):
    lines[idx] = _re1.sub(' and ', line)

In [65]:
for idx, line in enumerate(lines):
    line = re.sub(r'\-[\-]+', ' ', line, re.UNICODE)
    line = re.sub(r'\+[\+]+', ' ', line, re.UNICODE)
    line = re.sub(r'\_[\_]+', ' ', line, re.UNICODE)
    line = re.sub(r'\=[\=]+', ' ', line, re.UNICODE)
    lines[idx] = re.sub(r'\…[\…]+', ' ', line, re.UNICODE)

In [66]:
_re1 = re.compile(r'@[A-Za-z0-9_-]+(?=\s|\:|\.|\)|$)', re.UNICODE)


memo = {}

for line in lines:
    for match in _re1.finditer(line):
        if line[match.start():match.end()] not in list(memo.keys()):
            memo[line[match.start():match.end()]] = 1
        else:
            memo[line[match.start():match.end()]] += 1

for ix, line in enumerate(lines):
    words = line.split()
    for idx, word in enumerate(words):
        match = _re1.match(word)
        if match:
            if memo[word[match.start():match.end()]] <= 10:
                words[idx] = _re1.sub('@mention', word[match.start():match.end()])
    lines[ix] = ' '.join(words)

In [67]:
for idx, line in enumerate(lines):
    line = re.sub(r'@mention\s*@mention+', ' @mention ', line, re.UNICODE)
    lines[idx] = re.sub(r'\s\s+', ' ', line, re.UNICODE)

In [68]:
df[1] = lines
df.head()

,0,1
0,0,"@mention Liebe Corinna, wir würden dich gerne ..."
1,0,@mention Sie haben ja auch Recht. Unser Tweet ...
2,0,@mention fröhlicher gruß aus der schönsten sta...
3,0,@dushanwegner Amis hätten alles und jeden gewä...
4,1,@spdde kein verläßlicher Verhandlungspartner. ...


In [69]:
import numpy as np

shuff = np.random.rand(len(df)) < 0.8
_df = df[shuff]
df_ = df[~shuff]

_df.to_csv('train.csv', header=False, index=False)
df_.to_csv('val.csv', header=False, index=False)

In [55]:
df.to_csv('test.csv', header=False, index=False)